In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 12)

### Today = Last business day

In [2]:
#today = date(2023, 3, 13)
today

datetime.date(2023, 6, 12)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-12
yesterday: 2023-06-09


#### Restart and Run All

In [4]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [5]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1265677,TTB,2023-06-09,2023-06-09,0,1.66,1.66,0.00,0.00%,0,0,1.69,1.65,"403,425,915",2023-06-11,2023-06-11,1


In [6]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [7]:
name = names.to_string(index=False)
type(name)

str

In [8]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'TTB'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9717,TTB,0.00,0.00,O,1.09,0.00,0.00,0.00,0.00,"20,000",-9,17,0,0,0,52WL,SET50


### End of Tables in the process

In [9]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-06-09'
ORDER BY a.name



,name
0,AH
1,AWC
2,BANPU
3,BBL
4,BCH
5,BDMS
6,BEM
7,BH
8,CPNREIT
9,EA


In [10]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-06-09' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [11]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,"502,601,798",8.80,8.20,B,SET50,0.45,5.42%
1,IVL,2023-06-08,2023-06-09,33.25,33.25,"34,539,645",33.75,32.00,B,SET50,0.00,0.00%
2,MCS,2023-06-08,2023-06-09,7.00,7.00,"2,497,461",7.15,6.95,B,SET,0.00,0.00%
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,"22,419,726",11.60,10.70,B,SET100,0.00,0.00%
4,STA,2023-06-09,2023-06-09,18.80,18.80,"5,472,715",19.00,18.60,B,SET100,0.00,0.00%
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,"462,153",13.30,12.80,B,SET,0.30,2.31%
6,TMT,2023-06-09,2023-06-09,7.50,7.50,"239,238",7.50,7.45,B,SET,0.00,0.00%
7,AH,2023-06-08,2023-06-09,36.50,34.50,"103,830,264",37.75,27.75,I,SET,-2.00,-5.48%
8,BCH,2023-06-09,2023-06-09,18.30,18.30,"14,152,226",18.60,18.30,I,SET100,0.00,0.00%
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,"2,498,945",13.00,12.70,I,SET,0.00,0.00%


In [12]:
df.shape

(34, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [13]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,"502,601,798",8.80,8.20,5.42%,B
1,IVL,2023-06-08,2023-06-09,33.25,33.25,"34,539,645",33.75,32.00,0.00%,B
2,MCS,2023-06-08,2023-06-09,7.00,7.00,"2,497,461",7.15,6.95,0.00%,B
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,"22,419,726",11.60,10.70,0.00%,B
4,STA,2023-06-09,2023-06-09,18.80,18.80,"5,472,715",19.00,18.60,0.00%,B
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,"462,153",13.30,12.80,2.31%,B
6,TMT,2023-06-09,2023-06-09,7.50,7.50,"239,238",7.50,7.45,0.00%,B
7,AH,2023-06-08,2023-06-09,36.50,34.50,"103,830,264",37.75,27.75,-5.48%,I
8,BCH,2023-06-09,2023-06-09,18.30,18.30,"14,152,226",18.60,18.30,0.00%,I
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,"2,498,945",13.00,12.70,0.00%,I


In [14]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [15]:
df[cols].shape

(34, 10)

In [16]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,502601798,8.80,8.20,5.42,B
1,IVL,2023-06-08,2023-06-09,33.25,33.25,34539645,33.75,32.00,0.00,B
2,MCS,2023-06-08,2023-06-09,7.00,7.00,2497461,7.15,6.95,0.00,B
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,22419726,11.60,10.70,0.00,B
4,STA,2023-06-09,2023-06-09,18.80,18.80,5472715,19.00,18.60,0.00,B
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,462153,13.30,12.80,2.31,B
6,TMT,2023-06-09,2023-06-09,7.50,7.50,239238,7.50,7.45,0.00,B
7,AH,2023-06-08,2023-06-09,36.50,34.50,103830264,37.75,27.75,-5.48,I
8,BCH,2023-06-09,2023-06-09,18.30,18.30,14152226,18.60,18.30,0.00,I
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,2498945,13.00,12.70,0.00,I


In [17]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-06-12,1.99,0.01,0.505051,1.99,2.02,1.98,17077218,34095.85673
1,ADVANC,2023-06-12,218.00,-1.00,-0.456621,218.00,219.00,216.00,3391263,736748.12000
2,AEONTS,2023-06-12,202.00,1.00,0.497512,200.00,202.00,200.00,62902,12679.90200
3,AH,2023-06-12,35.25,0.75,2.173913,34.75,36.25,34.75,7101473,252533.77550
4,AIE,2023-06-12,1.50,0.00,0.000000,1.51,1.52,1.50,398959,602.04059
...,...,...,...,...,...,...,...,...,...,...
187,WHAIR,2023-06-12,7.25,-0.05,-0.684932,7.20,7.30,7.20,135404,977.91900
188,WHART,2023-06-12,11.20,0.20,1.818182,11.10,11.20,11.00,1052392,11689.76800
189,WHAUP,2023-06-12,3.86,0.02,0.520833,3.84,3.86,3.82,473970,1820.91418
190,WICE,2023-06-12,8.85,-0.05,-0.561798,8.95,8.95,8.75,1495003,13171.90105


In [18]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,502601798,8.80,8.20,5.42,B,2023-06-12,8.75,0.00,0.000000,8.80,8.80,8.65,46469654,4.051597e+05
1,IVL,2023-06-08,2023-06-09,33.25,33.25,34539645,33.75,32.00,0.00,B,2023-06-12,33.50,0.25,0.751880,33.25,33.75,33.00,10222011,3.403925e+05
2,MCS,2023-06-08,2023-06-09,7.00,7.00,2497461,7.15,6.95,0.00,B,2023-06-12,7.00,0.00,0.000000,7.00,7.05,6.95,153013,1.071176e+03
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,22419726,11.60,10.70,0.00,B,2023-06-12,11.10,-0.10,-0.892857,11.20,11.50,11.10,7953247,8.942528e+04
4,STA,2023-06-09,2023-06-09,18.80,18.80,5472715,19.00,18.60,0.00,B,2023-06-12,18.50,-0.30,-1.595745,18.80,18.80,18.40,4935652,9.177196e+04
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,462153,13.30,12.80,2.31,B,2023-06-12,13.20,-0.10,-0.751880,13.30,13.30,13.10,293569,3.882701e+03
6,TMT,2023-06-09,2023-06-09,7.50,7.50,239238,7.50,7.45,0.00,B,2023-06-12,7.45,-0.05,-0.666667,7.50,7.50,7.45,16180,1.208860e+02
7,AH,2023-06-08,2023-06-09,36.50,34.50,103830264,37.75,27.75,-5.48,I,2023-06-12,35.25,0.75,2.173913,34.75,36.25,34.75,7101473,2.525338e+05
8,BCH,2023-06-09,2023-06-09,18.30,18.30,14152226,18.60,18.30,0.00,I,2023-06-12,18.40,0.10,0.546448,18.40,18.50,18.30,7226526,1.328285e+05
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,2498945,13.00,12.70,0.00,I,2023-06-12,12.90,-0.10,-0.769231,12.90,13.00,12.80,360857,4.663182e+03


In [19]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [20]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,502601798,8.80,8.20,5.42,B,8.75,0.00,46469654,2023-06-12
1,IVL,2023-06-08,2023-06-09,33.25,33.25,34539645,33.75,32.00,0.00,B,33.50,0.25,10222011,2023-06-12
2,MCS,2023-06-08,2023-06-09,7.00,7.00,2497461,7.15,6.95,0.00,B,7.00,0.00,153013,2023-06-12
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,22419726,11.60,10.70,0.00,B,11.10,-0.10,7953247,2023-06-12
4,STA,2023-06-09,2023-06-09,18.80,18.80,5472715,19.00,18.60,0.00,B,18.50,-0.30,4935652,2023-06-12
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,462153,13.30,12.80,2.31,B,13.20,-0.10,293569,2023-06-12
6,TMT,2023-06-09,2023-06-09,7.50,7.50,239238,7.50,7.45,0.00,B,7.45,-0.05,16180,2023-06-12
7,AH,2023-06-08,2023-06-09,36.50,34.50,103830264,37.75,27.75,-5.48,I,35.25,0.75,7101473,2023-06-12
8,BCH,2023-06-09,2023-06-09,18.30,18.30,14152226,18.60,18.30,0.00,I,18.40,0.10,7226526,2023-06-12
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,2498945,13.00,12.70,0.00,I,12.90,-0.10,360857,2023-06-12


In [21]:
df_merge[colu].shape

(34, 14)

In [22]:
df_merge.shape

(34, 19)

In [23]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [24]:
### Call ruby ruby\daily-out-new.rb

In [25]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [26]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [27]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-06-12 2023-06-12  34.50  35.25   2.17% 103,830,264  37.75  27.75 I Insert
AWC      2023-06-09 2023-06-12   5.00   5.10    2.0%  15,460,325   5.10   5.00 O Update
BANPU    2023-06-02 2023-06-12   8.30   8.75   5.42% 502,601,798   8.80   8.20 B Update
BBL      2023-06-08 2023-06-12 163.00 163.00    0.0%  10,742,185 163.50 159.50 O Update
BCH      2023-06-09 2023-06-12  18.30  18.40   0.55%  14,152,226  18.60  18.30 I Update
BDMS     2023-06-09 2023-06-12  28.75  29.00   0.87%  58,670,443  29.00  28.25 O Update
BEM      2023-06-09 2023-06-12   8.35   8.35    0.0%  15,925,247   8.40   8.30 O Update
BH       2023-06-09 2023-06-12 235.00 232.00  -1.28%   2,113,062 238.00 232.00 O Update
CPNREIT  2023-06-09 2023-06-12  13.00  12.90  -0.77%   2,498,945  13.00  12.70 I Update
EA       2023-06-09 2023-06-12  

In [28]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb